In [67]:
import requests
import json
import pandas as pd

import time
from nba_api.stats.endpoints import playbyplayv2

In [ ]:
def load_datanba_pbp(game_id, season='2024', league_id='00'):
    if league_id == "00":
        url = f"https://data.nba.com/data/v2015/json/mobile_teams/nba/{season}/scores/pbp/{game_id}_full_pbp.json"
    else:
        url = f"https://data.wnba.com/data/v2015/json/mobile_teams/wnba/{season}/scores/pbp/{game_id}_full_pbp.json"

    headers = {
        "Connection": "keep-alive",
        "Accept": "application/json, text/plain, */*",
        "x-nba-stats-token": "true",
        "X-NewRelic-ID": "VQECWF5UChAHUlNTBwgBVw==",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36",
        "x-nba-stats-origin": "stats",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "cors",
        "Referer": "https://stats.nba.com",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
    }

    max_attempts = 5
    attempt = 0
    while attempt < max_attempts:
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            break  # successful request, exit loop
        except requests.RequestException:
            attempt += 1
            if attempt == max_attempts:
                raise Exception(f"No response received from data.nba.com after {max_attempts} attempts")
    
    game_id = data.get("g", {}).get("gid", "")
    periods = data.get("g", {}).get("pd", [])

    play_by_play = []
    for period in periods:
        plays = period.get("pla", [])
        for play in plays:
            play["PERIOD"] = period.get("p", "")
            play["GAME_ID"] = game_id
            play_by_play.append(play)

    return play_by_play

In [ ]:
def load_datanba_general(request_type="gameDetail", season="2024", gameDate="", gameId="", period=""):

  url_dict = {
    'leadTracker': f'http://data.nba.net/prod/v1/{gameDate}/{gameId}_lead_tracker_{period}.json',
    'playersPerGame': f'http://data.nba.net/json/cms/{season}/game/{gameDate}/{gameId}/playersPerGame.json',
    'gameDetail': f'http://data.nba.net/v2015/json/mobile_teams/nba/{season}/scores/gamedetail/{gameId}_gamedetail.json'
  }

  url = url_dict[request_type]
  print(url)

  headers = {
    "Connection": "keep-alive",
    "Accept": "application/json, text/plain, */*",
    "x-nba-stats-token": "true",
    "X-NewRelic-ID": "VQECWF5UChAHUlNTBwgBVw==",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36",
    "x-nba-stats-origin": "stats",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "cors",
    "Referer": "https://stats.nba.com",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
  }

  max_attempts = 5
  attempt = 0
  while attempt < max_attempts:
    try:
      response = requests.get(url, headers=headers, timeout=20)
      response.raise_for_status()
      data = response.json()
      break  # successful request, exit loop
    except requests.RequestException:
      print(requests.RequestException)
      time.sleep(5)
      attempt += 1
      if attempt == max_attempts:
        raise Exception(f"No response received from data.nba.com after {max_attempts} attempts")
  
  return data

In [66]:
test_data = load_datanba_general(gameId=gameID)

Exception: No response received from data.nba.com after 5 attempts

In [ ]:
gameID = '0022400061'
pbp_data = load_datanba(gameI)

{'evt': 4,
 'wallclk': '2024-10-22T23:35:57.9000000+00:00',
 'cl': '11:58',
 'de': 'Jump Ball Towns vs Towns (White gains possession)',
 'locX': 0,
 'locY': -80,
 'opt1': 0,
 'opt2': 0,
 'opt3': 0,
 'opt4': 0,
 'mtype': 0,
 'etype': 10,
 'opid': '1626157',
 'tid': 1610612738,
 'pid': 1626157,
 'hs': 0,
 'vs': 0,
 'epid': '1628401',
 'oftid': 1610612738,
 'ord': 60000,
 'pts': 0,
 'PERIOD': 1,
 'GAME_ID': '0022400061'}

In [11]:
df = pd.DataFrame(pbp_data)

In [15]:
df

,evt,wallclk,cl,de,locX,locY,opt1,opt2,opt3,opt4,...,tid,pid,hs,vs,epid,oftid,ord,pts,PERIOD,GAME_ID
0,2,2024-10-22T23:35:56.4000000+00:00,12:00,Start Period,0,-80,0,0,0,0,...,0,0,0,0,,0,20000,0,1,0022400061
1,4,2024-10-22T23:35:57.9000000+00:00,11:58,Jump Ball Towns vs Towns (White gains possession),0,-80,0,0,0,0,...,1610612738,1626157,0,0,1628401,1610612738,60000,0,1,0022400061
2,7,2024-10-22T23:36:07.5000000+00:00,11:48,[BOS 3-0] Tatum 3pt Shot: Made (3 PTS) Assist:...,141,229,3,0,0,0,...,1610612738,1628369,3,0,1628401,1610612738,70000,3,1,0022400061
3,9,2024-10-22T23:36:28.1000000+00:00,11:27,[NYK 2-3] Anunoby Driving Layup Shot: Made (2 ...,36,46,2,0,0,0,...,1610612752,1628384,3,2,1628973,1610612752,90000,2,1,0022400061
4,11,2024-10-22T23:36:38.5000000+00:00,11:16,[BOS] Tatum 3pt Shot: Missed,-83,242,3,0,0,0,...,1610612738,1628369,3,2,,1610612738,110000,0,1,0022400061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,546,2024-10-23T01:39:00.7000000+00:00,00:36.9,[NYK] Toppin 3pt Shot: Missed,223,110,3,0,0,0,...,1610612752,1631210,132,109,,1610612752,5320000,0,4,0022400061
393,547,2024-10-23T01:39:02.0000000+00:00,00:35.6,[BOS] Team Rebound,223,110,0,0,0,0,...,1610612738,0,132,109,,1610612752,5330000,0,4,0022400061
394,548,2024-10-23T01:39:07.0000000+00:00,00:35.6,Stoppage: Out-of-Bounds,0,-80,0,0,0,0,...,0,0,132,109,,1610612738,5340000,0,4,0022400061
395,549,2024-10-23T01:39:38.7000000+00:00,00:11.0,[BOS] Team Turnover : Shot Clock Turnover,167,373,0,0,0,0,...,1610612738,0,132,109,200834,1610612738,5350000,0,4,0022400061


In [17]:
game_pbp = playbyplayv2.PlayByPlayV2(game_id=gameID).get_data_frames()[0]

In [22]:
join_df = game_pbp.merge(df, left_on='EVENTNUM', right_on='evt')

In [23]:
join_df.columns

Index(['GAME_ID_x', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE',
       'PERIOD_x', 'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION',
       'NEUTRALDESCRIPTION', 'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN',
       'PERSON1TYPE', 'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID',
       'PLAYER1_TEAM_CITY', 'PLAYER1_TEAM_NICKNAME',
       'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE', 'PLAYER2_ID',
       'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION',
       'VIDEO_AVAILABLE_FLAG', 'evt', 'wallclk', 'cl', 'de', 'locX', 'locY',
       'opt1', 'opt2', 'opt3', 'opt4', 'mtype', 'etype', 'opid', 'tid', 'pid',
       'hs', 'vs', 'epid', 'oftid', 'ord', 'pts', 'PERIOD_y', 'GAME_ID_y'],
      dtype='object')

In [34]:
join_df[[col for col in join_df.columns if 'DE' in col or 'de' in col]]

,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,VIDEO_AVAILABLE_FLAG,de
0,None,Start of 1st Period (7:36 PM EST),None,0,Start Period
1,Jump Ball Horford vs. Towns: Tip to White,None,None,1,Jump Ball Towns vs Towns (White gains possession)
2,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,None,None,1,[BOS 3-0] Tatum 3pt Shot: Made (3 PTS) Assist:...
3,None,None,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST),1,[NYK 2-3] Anunoby Driving Layup Shot: Made (2 ...
4,MISS Tatum 26' 3PT Pullup Jump Shot,None,None,1,[BOS] Tatum 3pt Shot: Missed
...,...,...,...,...,...
383,None,None,Knicks Rebound,0,[NYK] Team Rebound
384,None,None,MISS Toppin 25' 3PT Jump Shot,1,[NYK] Toppin 3pt Shot: Missed
385,CELTICS Rebound,None,None,0,[BOS] Team Rebound
386,CELTICS Turnover: Shot Clock (T#4),None,None,1,[BOS] Team Turnover : Shot Clock Turnover


In [54]:
df[df.opt2 == 1].de

6      [NYK 4-3] Brunson Running Layup Shot: Made (2 ...
20                [BOS] White Running Layup Shot: Missed
23     [BOS 13-10] Tatum Running Dunk Shot: Made (5 P...
70                          [BOS] White 3pt Shot: Missed
83     [BOS 32-17] Holiday Running Layup Shot: Made (...
109              [BOS] Hauser Running Layup Shot: Missed
122                         [BOS] Brown 3pt Shot: Missed
139                       [NYK] Anunoby 3pt Shot: Missed
162    [BOS 59-37] Brown Running Dunk Shot: Made (10 ...
216          [BOS 83-57] Holiday 3pt Shot: Made (13 PTS)
237    [NYK 68-93] Brunson Running Layup Shot: Made (...
281                         [BOS] White 3pt Shot: Missed
320    [BOS 128-93] Holiday Running Layup Shot: Made ...
334                         [NYK] Payne 3pt Shot: Missed
Name: de, dtype: object

In [39]:
join_df.ord

0        20000
1        60000
2        70000
3        90000
4       110000
        ...   
383    5300000
384    5320000
385    5330000
386    5350000
387    5360000
Name: ord, Length: 388, dtype: int64

In [57]:
from nba_api.stats.endpoints import playbyplayv3

ImportError: cannot import name 'playbyplayv3' from 'nba_api.stats.endpoints' (/Users/benporter/opt/miniconda3/lib/python3.8/site-packages/nba_api/stats/endpoints/__init__.py)